# The Barnyard problem

An example of solving a well-behaved set of linear equations by Gaussian Elimination

JMA 28 Dec 2021

In [4]:
import os, sys
import numpy as np
import scipy.linalg    # A case where the top level package needs it's subpackages imported explicitly. 
sys.version

'3.10.1 (v3.10.1:2cd268a3a9, Dec  6 2021, 14:28:59) [Clang 13.0.0 (clang-1300.0.29.3)]'

In [5]:
# The barnyard variables

A = np.array([[1,1,1], [2,4,4], [0,1,2]])
b = np.transpose(np.array([[12, 38,10]]))

In [6]:
Ab = np.concatenate([A,b], axis=1)
Ab

array([[ 1,  1,  1, 12],
       [ 2,  4,  4, 38],
       [ 0,  1,  2, 10]])

## Elimination

In [7]:
def pivot_one_element(row, by_row, coef, matrix ):
    'Add coef times by_row to row of matrix. Indicies are zero-based.'
    new_row = matrix[row,:] + coef * matrix[by_row,:]
    # A copy is necessary since the matrix assignment is destructive, 
    # and its value is returned by reference. 
    m_copy = matrix.copy()
    m_copy[row,:]  = new_row
    return m_copy

Ab_1 = pivot_one_element(1, 0, -2, Ab)
Ab_1

array([[ 1,  1,  1, 12],
       [ 0,  2,  2, 14],
       [ 0,  1,  2, 10]])

In [8]:
# Use a pivot step to normalize the second pivot.
Ab_2 = pivot_one_element(1, 1, -1/2, Ab_1)
Ab_2

array([[ 1,  1,  1, 12],
       [ 0,  1,  1,  7],
       [ 0,  1,  2, 10]])

In [9]:
Uc = pivot_one_element(2, 1, -1, Ab_2)
Uc

array([[ 1,  1,  1, 12],
       [ 0,  1,  1,  7],
       [ 0,  0,  1,  3]])

## Back substitution

In [10]:
def back_substitute_step(U_matrix, solved_rows = []):
    'Using the values from the solved rows, return the solved rows list with the next value.'
    # Assume pivots have been normalized to 1
    row_cnt = U_matrix.shape[0] -1
    col_cnt = U_matrix.shape[1] -1
    row_to_solve = U_matrix[row_cnt - len(solved_rows),:]
    b = row_to_solve[-1]
    backfill_values = row_to_solve[(-len(solved_rows)-1):-1]
    next_value = (b - backfill_values.dot(solved_rows))
    return [next_value] + solved_rows

In [11]:
# First, solve a step at a time
# The last row back_substitution
x3 = back_substitute_step(Uc)
x3

[3]

In [12]:
# Continue
x2 = back_substitute_step(Uc, x3)
x2

[4, 3]

In [13]:
# Solution
x = back_substitute_step(Uc, x2)
x

[5, 4, 3]

In [14]:
## Solve by substituting all rows.
def solve_back_substitution(U_matrix):
    solution = []
    while len(solution) < U_matrix.shape[0]:
        solution = back_substitute_step(U_matrix, solution)
    return solution

solve_back_substitution(Uc)

[5, 4, 3]

## Automate all pivots

In [15]:
def normalize_a_row(the_row, the_matrix):
    'Assume the pivot has been made, so that the diagonal element on the_row will be set to 1'
    pivot = the_matrix[the_row,the_row]
    matrix_copy = the_matrix.copy()
    normal_row = matrix_copy[the_row,:]
    if pivot != 1:
        if abs(pivot) > 1E-7:
            #matrix_copy[the_row,:]
            normal_row = matrix_copy[the_row,:]/pivot
        else:
            print('Warning: zero  pivot value, cannot normalize')
    return normal_row

In [16]:
# Find the next pivots 
def next_pivots(pivot_this_row, the_matrix):
    'Zero out all entries in rows below this pivot.'
    matrix_copy = the_matrix.copy()
    the_row = matrix_copy[pivot_this_row,:]
    for a_row_index in range(pivot_this_row+1, the_matrix.shape[0]):
        coef = - matrix_copy[a_row_index, pivot_this_row] / the_row[pivot_this_row]
        print(a_row_index, coef)
        nr = pivot_one_element(a_row_index, pivot_this_row, coef, matrix_copy )
        print(nr)
        matrix_copy[a_row_index,:] = normalize_a_row(a_row_index, nr)
    return matrix_copy
        
next_pivots(0, Ab)

1 -2.0
[[ 1  1  1 12]
 [ 0  2  2 14]
 [ 0  1  2 10]]
2 0.0
[[ 1  1  1 12]
 [ 0  1  1  7]
 [ 0  1  2 10]]


array([[ 1,  1,  1, 12],
       [ 0,  1,  1,  7],
       [ 0,  0,  1,  5]])

In [17]:
# Solve for U by running all pivots
def solve_for_U(A_matrix):
    ''
    matrix_copy = A_matrix.copy()
    for row_index in range(0, A_matrix.shape[0]):
        matrix_copy = next_pivots(row_index, matrix_copy)
    return matrix_copy
    
solve_for_U(Ab)

1 -2.0
[[ 1  1  1 12]
 [ 0  2  2 14]
 [ 0  1  2 10]]
2 0.0
[[ 1  1  1 12]
 [ 0  1  1  7]
 [ 0  1  2 10]]
2 0.0
[[ 1  1  1 12]
 [ 0  1  1  7]
 [ 0  0  1  5]]


array([[ 1,  1,  1, 12],
       [ 0,  1,  1,  7],
       [ 0,  0,  1,  5]])

# Use the built-in numpy solver

Note, numpy does not bother with an LU decomposition, but computes the inverse directly

In [18]:
A_inverse = np.linalg.inv(A)
print('Inverse:\n', A_inverse,'\n\nSolution:')
A_inverse.dot(b)

Inverse:
 [[ 2.  -0.5  0. ]
 [-2.   1.  -1. ]
 [ 1.  -0.5  1. ]] 

Solution:


array([[5.],
       [4.],
       [3.]])

In [19]:
# Alternately:
np.linalg.solve(A,b)

array([[5.],
       [4.],
       [3.]])

In [20]:
# Use the LU decomposition

P, L, U = scipy.linalg.lu(A)

In [21]:
P @ A == L @ U   # @-sign is short for matrix multiplication. 

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [22]:
# What are the pivots?
L, np.linalg.inv(L)

(array([[ 1. ,  0. ,  0. ],
        [ 0.5,  1. ,  0. ],
        [ 0. , -1. ,  1. ]]),
 array([[ 1. ,  0. ,  0. ],
        [-0.5,  1. ,  0. ],
        [-0.5,  1. ,  1. ]]))

In [26]:
# 
np.linalg.inv(U) @ np.linalg.inv(L)

array([[-0.5,  2. ,  0. ],
       [ 1. , -2. , -1. ],
       [-0.5,  1. ,  1. ]])

In [23]:
# Two step solution with LU: 
# Step one - apply row transformations to b
y = np.linalg.inv(L) @ P @ b
y

array([[38.],
       [-7.],
       [ 3.]])

In [24]:
# Step two -- back substitution. 
np.linalg.inv(U) @ y

array([[5.],
       [4.],
       [3.]])